In [ ]:
import pygame
import sqlite3
import random
import sys

#настройки
WINDOW_WIDTH = 600
WINDOW_HEIGHT = 400
BLOCK_SIZE = 20
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (200, 0, 0)
BLACK = (0, 0, 0)

#база данных
def connect_db():
    return sqlite3.connect('snake_game.db')

def create_tables():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS user (
                        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE NOT NULL)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS user_score (
                        score_id INTEGER PRIMARY KEY AUTOINCREMENT,
                        user_id INTEGER NOT NULL,
                        level INTEGER NOT NULL,
                        score INTEGER NOT NULL,
                        saved_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        FOREIGN KEY(user_id) REFERENCES user(user_id))''')
    conn.commit()
    conn.close()

def get_or_create_user(username):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT user_id FROM user WHERE username=?", (username,))
    user = cursor.fetchone()
    if user:
        return user[0]
    cursor.execute("INSERT INTO user (username) VALUES (?)", (username,))
    conn.commit()
    return cursor.lastrowid

def get_last_level(user_id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT level FROM user_score WHERE user_id=? ORDER BY saved_at DESC LIMIT 1", (user_id,))
    row = cursor.fetchone()
    conn.close()
    return row[0] if row else 1

def save_game_state(user_id, level, score):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO user_score (user_id, level, score) VALUES (?, ?, ?)", (user_id, level, score))
    conn.commit()
    conn.close()

#уровни
levels_data = {
    1: {"speed": 10, "walls": []},
    2: {"speed": 12, "walls": [(100, 100), (120, 100), (140, 100)]},
    3: {"speed": 15, "walls": [(200, y) for y in range(0, 200, 20)]},
    4: {"speed": 18, "walls": [(x, 300) for x in range(100, 400, 20)]},
}

def get_level_by_score(score):
    if score < 30:
        return 1
    elif score < 60:
        return 2
    elif score < 100:
        return 3
    else:
        return 4

#отрисовка
def draw_snake(screen, snake_list):
    for block in snake_list:
        pygame.draw.rect(screen, GREEN, [block[0], block[1], BLOCK_SIZE, BLOCK_SIZE])

def draw_food(screen, food_x, food_y):
    pygame.draw.rect(screen, RED, [food_x, food_y, BLOCK_SIZE, BLOCK_SIZE])

#мэйн игра
def game_loop(user_id, starting_level):
    pygame.init()
    screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
    pygame.display.set_caption("🐍 Snake Game with Levels")
    clock = pygame.time.Clock()

    x, y = WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2
    dx, dy = BLOCK_SIZE, 0

    snake = []
    length = 1
    score = 0

    food_x = random.randrange(0, WINDOW_WIDTH, BLOCK_SIZE)
    food_y = random.randrange(0, WINDOW_HEIGHT, BLOCK_SIZE)

    run = True
    paused = False

    while run:
        level = get_level_by_score(score)
        speed = levels_data[level]["speed"]
        walls = levels_data[level]["walls"]

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT and dx == 0:
                    dx, dy = -BLOCK_SIZE, 0
                elif event.key == pygame.K_RIGHT and dx == 0:
                    dx, dy = BLOCK_SIZE, 0
                elif event.key == pygame.K_UP and dy == 0:
                    dx, dy = 0, -BLOCK_SIZE
                elif event.key == pygame.K_DOWN and dy == 0:
                    dx, dy = 0, BLOCK_SIZE
                elif event.key == pygame.K_p:
                    save_game_state(user_id, level, score)
                    paused = not paused

        if paused:
            continue

        x += dx
        y += dy

        if x < 0 or x >= WINDOW_WIDTH or y < 0 or y >= WINDOW_HEIGHT or [x, y] in snake or (x, y) in walls:
            run = False

        screen.fill(BLACK)

        snake.append([x, y])
        if len(snake) > length:
            del snake[0]

        if x == food_x and y == food_y:
            food_x = random.randrange(0, WINDOW_WIDTH, BLOCK_SIZE)
            food_y = random.randrange(0, WINDOW_HEIGHT, BLOCK_SIZE)
            length += 1
            score += 10

        draw_food(screen, food_x, food_y)

        for wall in walls:
            pygame.draw.rect(screen, WHITE, [wall[0], wall[1], BLOCK_SIZE, BLOCK_SIZE])

        draw_snake(screen, snake)

        pygame.display.update()
        clock.tick(speed)

    save_game_state(user_id, level, score)
    print(f"\nИгра окончена. Ты достиг уровня: {level}, твой счёт: {score}")
    pygame.quit()
    sys.exit()


#запсук
if __name__ == "__main__":
    create_tables()
    username = input("Введите имя пользователя: ")
    user_id = get_or_create_user(username)
    last_level = get_last_level(user_id)
    print(f"Добро пожаловать, {username}! Начнем с уровня {last_level}")
    game_loop(user_id, last_level)